<a href="https://colab.research.google.com/github/AnastasiaBrinati/Progetto-ML-23-24/blob/main/task3_centralizzato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
!pip install -q -U keras-tuner
import keras_tuner as kt
from sklearn.metrics import precision_score, recall_score, f1_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [3]:
# Inizializza il seed per NumPy per riproducibilità
np.random.seed(42)

# Inizializza il seed per TensorFlow per riproducibilità
tf.random.set_seed(42)
# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocessing
# Reshape and scale the input features
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# One-hot encode the target labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# Define the model builder function for hyperparameter tuning
def model_builder(hp):
    model = keras.Sequential()

    # Tune the number of convolutional layers
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(layers.Conv2D(
            filters=hp.Int(f'filters_{i}', min_value=16, max_value=128, step=16),
            kernel_size=hp.Choice(f'kernel_size_{i}', values=[3, 5]),
            activation='relu',
            padding='same'
        ))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())

    # Tune the number of units and activation function for each Dense layer
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=16, max_value=128, step=16),
            activation=hp.Choice(f'activation_{i}', values=['PReLU','LeakyReLU','relu', 'tanh', 'sigmoid'])
        ))

    # Output layer
    model.add(layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=["accuracy", "categorical_accuracy", "top_k_categorical_accuracy"])

    return model

# Initialize the tuner
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='mnist_tuning')

# Perform hyperparameter search
tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model with the optimal hyperparameters
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=0)

# Evaluate the model on test data
test_loss, test_accuracy, test_categorical_accuracy, test_top_k_categorical_accuracy = model.evaluate(X_test, y_test)

# Predictions on the validation set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate precision, recall, and F1-score
test_precision = precision_score(np.argmax(y_test, axis=1), y_pred_classes, average='weighted')
test_recall = recall_score(np.argmax(y_test, axis=1), y_pred_classes, average='weighted')
test_f1_score = f1_score(np.argmax(y_test, axis=1), y_pred_classes, average='weighted')

# Display metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test Categorical Accuracy:", test_categorical_accuracy)
print("Test Top-K Categorical Accuracy:", test_top_k_categorical_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1_score)


Trial 30 Complete [00h 23m 13s]
val_accuracy: 0.9601666927337646

Best val_accuracy So Far: 0.9915000200271606
Total elapsed time: 05h 32m 32s
313/313 [==============================] - 16s 51ms/step
Test Loss: 0.06348137557506561
Test Accuracy: 0.9857000112533569
Test Categorical Accuracy: 0.9857000112533569
Test Top-K Categorical Accuracy: 0.9995999932289124
Test Precision: 0.9859502247152615
Test Recall: 0.9857
Test F1 Score: 0.985740145638161
